In [32]:
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
import IPython.display as ipd
import librosa
import librosa.display
audio_file_path=filename
librosa_audio_data,librosa_sample_rate=librosa.load(audio_file_path)

In [34]:
import pandas as pd

metadata=pd.read_csv('CSV\metadata.csv')
metadata.head(10)

,#,file_name,class_ID,class
0,1,spacek_1.wav,1,spacek
1,2,spacek_10.wav,1,spacek
2,3,spacek_100.wav,1,spacek
3,4,spacek_11.wav,1,spacek
4,5,spacek_12.wav,1,spacek
5,6,spacek_13.wav,1,spacek
6,7,spacek_14.wav,1,spacek
7,8,spacek_15.wav,1,spacek
8,9,spacek_16.wav,1,spacek
9,10,spacek_17.wav,1,spacek


In [35]:
mfccs = librosa.feature.mfcc(y=librosa_audio_data, sr=librosa_sample_rate, n_mfcc=40)
print(mfccs.shape)

(40, 431)


In [36]:
import pandas as pd
import os
import librosa

audio_dataset_path='spacek_pole/'
metadata=pd.read_csv('CSV\metadata.csv')
metadata.head()

,#,file_name,class_ID,class
0,1,spacek_1.wav,1,spacek
1,2,spacek_10.wav,1,spacek
2,3,spacek_100.wav,1,spacek
3,4,spacek_11.wav,1,spacek
4,5,spacek_12.wav,1,spacek


In [37]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [38]:
import numpy as np
from tqdm import tqdm
### Now we iterate through every audio file and extract features 
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),str(row["file_name"]))
    final_class_labels=row["class"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

200it [00:18, 10.71it/s]


In [39]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])

In [40]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [41]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [42]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [43]:
import tensorflow as tf
print(tf.__version__)

2.8.0


In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [45]:
### No of classes
num_labels=y.shape[1]

model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [46]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               4100      
                                                                 
 activation_4 (Activation)   (None, 100)               0         
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 200)               20200     
                                                                 
 activation_5 (Activation)   (None, 200)               0         
                                                                 
 dropout_4 (Dropout)         (None, 200)               0         
                                                                 
 dense_6 (Dense)             (None, 100)              

In [47]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [48]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/100
1/5 [=====>........................] - ETA: 1s - loss: 23.6725 - accuracy: 0.4688
Epoch 1: val_loss improved from inf to 5.44632, saving model to saved_models\audio_classification.hdf5
5/5 [==============================] - 0s 37ms/step - loss: 19.1943 - accuracy: 0.5375 - val_loss: 5.4463 - val_accuracy: 0.5250
Epoch 2/100
1/5 [=====>........................] - ETA: 0s - loss: 13.5905 - accuracy: 0.5312
Epoch 2: val_loss improved from 5.44632 to 1.54019, saving model to saved_models\audio_classification.hdf5
5/5 [==============================] - 0s 11ms/step - loss: 13.0965 - accuracy: 0.6375 - val_loss: 1.5402 - val_accuracy: 0.6750
Epoch 3/100
1/5 [=====>........................] - ETA: 0s - loss: 9.6339 - accuracy: 0.5312
Epoch 3: val_loss improved from 1.54019 to 0.00758, saving model to saved_models\audio_classification.hdf5
5/5 [==============================] - 0s 12ms/step - loss: 8.1118 - accuracy: 0.6812 - val_loss: 0.0076 - val_accuracy: 1.0000
Epoch 4/100
1/5 

Epoch 28/100
1/5 [=====>........................] - ETA: 0s - loss: 0.8180 - accuracy: 0.9375
Epoch 28: val_loss did not improve from 0.00000
5/5 [==============================] - 0s 6ms/step - loss: 0.9445 - accuracy: 0.9250 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 29/100
1/5 [=====>........................] - ETA: 0s - loss: 0.0089 - accuracy: 1.0000
Epoch 29: val_loss did not improve from 0.00000
5/5 [==============================] - 0s 6ms/step - loss: 0.3823 - accuracy: 0.9812 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 30/100
1/5 [=====>........................] - ETA: 0s - loss: 0.4042 - accuracy: 0.9375
Epoch 30: val_loss did not improve from 0.00000
5/5 [==============================] - 0s 6ms/step - loss: 0.4456 - accuracy: 0.9625 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 31/100
1/5 [=====>........................] - ETA: 0s - loss: 0.3537 - accuracy: 0.9688
Epoch 31: val_loss did not improve from 0.00000
5/5 [==============================] - 

Epoch 58/100
1/5 [=====>........................] - ETA: 0s - loss: 1.2852e-06 - accuracy: 1.0000
Epoch 58: val_loss did not improve from 0.00000
5/5 [==============================] - 0s 5ms/step - loss: 0.1138 - accuracy: 0.9812 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 59/100
1/5 [=====>........................] - ETA: 0s - loss: 0.1776 - accuracy: 0.9688
Epoch 59: val_loss did not improve from 0.00000
5/5 [==============================] - 0s 6ms/step - loss: 0.0356 - accuracy: 0.9937 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 60/100
1/5 [=====>........................] - ETA: 0s - loss: 0.0134 - accuracy: 1.0000
Epoch 60: val_loss did not improve from 0.00000
5/5 [==============================] - 0s 5ms/step - loss: 0.3472 - accuracy: 0.9875 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 61/100
1/5 [=====>........................] - ETA: 0s - loss: 0.0082 - accuracy: 1.0000
Epoch 61: val_loss did not improve from 0.00000
5/5 [==============================

Epoch 88/100
1/5 [=====>........................] - ETA: 0s - loss: 2.0672e-05 - accuracy: 1.0000
Epoch 88: val_loss did not improve from 0.00000
5/5 [==============================] - 0s 5ms/step - loss: 0.0165 - accuracy: 0.9937 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 89/100
1/5 [=====>........................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 89: val_loss did not improve from 0.00000
5/5 [==============================] - 0s 5ms/step - loss: 0.0343 - accuracy: 0.9937 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 90/100
1/5 [=====>........................] - ETA: 0s - loss: 0.0023 - accuracy: 1.0000
Epoch 90: val_loss did not improve from 0.00000
5/5 [==============================] - 0s 5ms/step - loss: 0.1649 - accuracy: 0.9875 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 91/100
1/5 [=====>........................] - ETA: 0s - loss: 1.8924e-06 - accuracy: 1.0000
Epoch 91: val_loss did not improve from 0.00000
5/5 [======================

In [18]:
model.save('savedModel.bin')

INFO:tensorflow:Assets written to: savedModel.bin\assets


In [49]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])
#velmi vysoka uspesnost kvuli malemu objemu dat v datasetu

1.0


In [50]:
filename="spacek_pole/pole_1.wav"
def vyhodnot(filename):
    audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

    print(mfccs_scaled_features)
    mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
    print(mfccs_scaled_features)
    print(mfccs_scaled_features.shape)
    predicted_label=model.predict(mfccs_scaled_features)
    #predicted_label[0]=[round(predicted_label[0]),round(predicted_label[1])]
    print(predicted_label)
    #prediction_class = labelencoder.inverse_transform(predicted_label) 
    #prediction_class
    print('spacek' if predicted_label[0][1] > predicted_label[0][0] else 'neco jineho')

vyhodnot(filename)

[-270.9278      149.16931      24.035181     41.094627      4.3992357
   24.285896      4.479779      8.0093975    -2.296018      5.5685644
   -7.3798356    -0.56679416   -7.9977593    -3.0547051    -5.810513
   -3.3426561    -6.1915264    -3.3708768    -2.932402     -2.6819296
   -2.8071358    -2.1917636    -3.4300292    -3.61432      -3.0705793
   -4.283662     -3.775552     -4.470292     -5.600348     -5.5041237
   -5.112518     -4.681289     -5.559829     -4.574327     -3.858078
   -5.237159     -3.477666     -4.792389     -4.183368     -4.291699  ]
[[-270.9278      149.16931      24.035181     41.094627      4.3992357
    24.285896      4.479779      8.0093975    -2.296018      5.5685644
    -7.3798356    -0.56679416   -7.9977593    -3.0547051    -5.810513
    -3.3426561    -6.1915264    -3.3708768    -2.932402     -2.6819296
    -2.8071358    -2.1917636    -3.4300292    -3.61432      -3.0705793
    -4.283662     -3.775552     -4.470292     -5.600348     -5.5041237
    -5.112518  